<a href="https://colab.research.google.com/github/varunreddyGOPU/AI-_Image_Generation-for-Product-Marketing/blob/main/Image_gen_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

# Initialize the client with your API key
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

# Define your prompt
prompt = "image of fruits and milk with safeway logo"

# Generate the image
response = client.models.generate_images(
    model='imagen-3.0-generate-002',
    prompt=prompt,
    config=types.GenerateImagesConfig(number_of_images=1)
)

for generated_image in response.generated_images:
    # Get the generated image
    image = Image.open(BytesIO(generated_image.image.image_bytes))

    # Open the logo you want to add (make sure it's in your working directory)
    logo = Image.open("/content/sample_data/albertsons-logo-1.png")
    # Resize the logo if needed (example: set width to 100px, keep aspect ratio)
    logo_width = 100
    logo_height = int(logo.height * (logo_width / logo.width))
    logo = logo.resize((logo_width, logo_height))

    # Paste the logo onto the generated image (adjust x, y position as needed)
    image.paste(logo, (image.width - logo.width - 10, image.height - logo.height - 10), logo)

    # Show the image (may not work in all environments)
    image.show()
    # Save the image with the logo
    image.save("output2_with_logo.png")

In [18]:
import requests
from google.colab import userdata

SONAR_API_KEY = userdata.get('SONAR_API_KEY')
SONAR_API_URL = "https://api.perplexity.ai/chat/completions"

headers = {
    "Authorization": f"Bearer {SONAR_API_KEY}",
    "Content-Type": "application/json"
}

prompt = """
Give a description about eating healthy with strawberries, oranges, and bananas with milk.
Include information about the main nutrient values in these foods.
"""

data = {
    "model": "sonar-pro",
    "messages": [
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(SONAR_API_URL, headers=headers, json=data)
print("Status code:", response.status_code)
print("Response text:", response.text)

if response.status_code == 200:
    try:
        # Check the actual response structure; may be "choices[0].message.content" or similar
        description = response.json()["choices"][0]["message"]["content"]
        print(description)
    except Exception as e:
        print("Error parsing response:", e)
else:
    print("Failed to get description:", response.text)


Status code: 200
Response text: {"id": "c5d2c09f-782e-4d07-8bc2-cb38b74093cf", "model": "sonar-pro", "created": 1748435627, "usage": {"prompt_tokens": 27, "completion_tokens": 554, "total_tokens": 581, "search_context_size": "low"}, "citations": ["https://www.allrecipes.com/recipe/215163/strawberry-orange-banana-smoothie/", "https://www.ambitiouskitchen.com/strawberry-orange-banana-smoothie/", "https://www.myfooddiary.com/foods/7378467/great-value-strawberry-orange-banana-drink-mix", "https://www.ewg.org/foodscores/products/043000950548-CrystalLightDrinkMixStrawberryOrangeBanana/", "https://smoothiebox.com/blogs/information/strawberry-banana-smoothie-guide"], "object": "chat.completion", "choices": [{"index": 0, "finish_reason": "stop", "message": {"role": "assistant", "content": "Eating healthy with strawberries, oranges, bananas, and milk creates a nutrient-dense combination that offers numerous health benefits. These ingredients work wonderfully together, especially in smoothies, pr